In [71]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from statistics import mean, variance
from functools import lru_cache
from math import sqrt
from typing import List, Tuple
import itertools
import json

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
#Initializing parameters
input_shape = (28, 28, 1)
batch_size = 64
num_classes = 10
epochs = 5

#Cleaning data
x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

2025-01-08 16:04:39.630788: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
def make_model(
    cnn_layer_sizes : List[Tuple[int, Tuple[int, int]]],
    dense_layer_sizes : List[int],
    activation_function : str
):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(input_shape))
    for i in range(len(cnn_layer_sizes)-1):
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[i][0], cnn_layer_sizes[i][1], activation=activation_function))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    if len(cnn_layer_sizes) > 0:
        model.add(tf.keras.layers.Conv2D(cnn_layer_sizes[-1][0], cnn_layer_sizes[-1][1], activation=activation_function))
    model.add(tf.keras.layers.Flatten())
    for elem in dense_layer_sizes:
        model.add(tf.keras.layers.Dense(elem, activation = activation_function))
    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
    model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])
    return model

def find_screens_that_mess_with_model(model):
    pass

In [32]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.995):
            print("\nReached 99.5% accuracy so cancelling training!")
            self.modelel.stop_training = True

callbacks = myCallback()

model = make_model([(10, (5,5)), (5, (3,3))], [64, 64], 'tanh')

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=5,
                    validation_split=0.1,
                    callbacks=[callbacks])

Epoch 1/5
844/844 [==============================] - 5s 5ms/step - loss: 0.2601 - acc: 0.9242 - val_loss: 0.0822 - val_acc: 0.9770
Epoch 2/5
844/844 [==============================] - 4s 5ms/step - loss: 0.0852 - acc: 0.9752 - val_loss: 0.0606 - val_acc: 0.9818
Epoch 3/5
844/844 [==============================] - 4s 5ms/step - loss: 0.0586 - acc: 0.9818 - val_loss: 0.0632 - val_acc: 0.9832
Epoch 4/5
844/844 [==============================] - 4s 5ms/step - loss: 0.0450 - acc: 0.9863 - val_loss: 0.0583 - val_acc: 0.9858
Epoch 5/5
844/844 [==============================] - 4s 5ms/step - loss: 0.0365 - acc: 0.9884 - val_loss: 0.0687 - val_acc: 0.9810


In [45]:
model.get_config()
#history.history['acc'][-1]

{'name': 'sequential_15',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_13'}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d_15',
    'trainable': True,
    'dtype': 'float32',
    'filters': 10,
    'kernel_size': (5, 5),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'groups': 1,
    'activation': 'tanh',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'MaxPooling2D',
   'config': {'name': 'max_pooling2d_4',
    'trainable': True,
    'dtype': 'float32',
    'pool_size': (2, 2)

In [66]:
#CONFIGURATIONS
cnn_layer_depth = [1,2,3]
cnn_layer_dimensions = [(2,2), (3,3), (4,4), (5,5)]
num_cnn_layers = [1,2,3]
num_dense_layers = [1,2,3,4]
dense_layer_sizes = [16, 32, 64, 128, 256]
activation_functions = ['relu', 'sigmoid', 'tanh']
epochs = [1,2,3,4,5]

def gen_cnn_layer_combinations():
    L = []
    for layer in num_cnn_layers:
        for dim in cnn_layer_dimensions:
            L.append((layer, dim))
    return L

def iterate_combinations():
    counter = 0
    pbar = tqdm(total = 1884 * 780 * 3 * 5)
    for cnn_depth in cnn_layer_depth:
        for cnn_layer_dim in itertools.product(gen_cnn_layer_combinations(), repeat=cnn_depth):
            for dense_depth in num_dense_layers:
                for dense_layers in itertools.product(dense_layer_sizes, repeat=dense_depth):
                    for a_func in activation_functions:
                        for epoch in epochs:
                            callbacks = myCallback()
                            model = make_model(list(cnn_layer_dim), list(dense_layers), a_func)
                            history = model.fit(x_train, y_train,
                                batch_size=batch_size,
                                epochs=epoch,
                                validation_split=0.1,
                                callbacks=[callbacks])
                            filename = f'model_configs/model{counter}.json'
                            model.save(filename)
                            counter += 1
                            pbar.update(1)
                            yield model, filename, history.history['acc'][-1]

In [70]:
counter = 0
for i in num_dense_layers:
    for v in itertools.product(dense_layer_sizes, repeat=i):
        counter += 1
counter

780

In [58]:
def gen_cnn_layer_combinations():
    L = []
    for layer in num_cnn_layers:
        for dim in cnn_layer_dimensions:
            L.append([layer, dim])
    return L